# Miary siły kolokacji

Faktem jest, że niektóre słowa występują z pewnymi słowami częściej, a z innymi radziej. Dla przykładu - fraza "mocna herbata" występować będzie w materiale korpusowym znacznie częściej niż "silna herbata" bądź "potężna herbata". Identyfikacja takich związków między słowami (wyrażeniami, jednostkami semantycznymi) jest problemem, z którym mierzyć się muszą rozmaite dziedziny. Dla niektórych (tzw. szkoła neo-Firthowska) znalezienie kolokatów danego słowa pozwala zbudować tzw. rozszerzoną jednostkę leksykalną, w której oprócz prostego znaczenia słowa znajdują się również jego wzorce użycia, informacje o łączliwości z pewnymi kategoriami semantycznymi itp. Innym zastosowaniem jest ekstrakcja ze specjalistycznych tekstów pewnych technicznych zwrotów, które powinny być zawsze tłumaczone w taki sam sposób, np. "internal combustion engine".

Językoznawcy, inżynierowie i statystycy opracowali kilka miar, za pomocą których możemy zmierzyć siłę kolokacji między dwoma słowami. Podstawowa idea, na której opiera się większość z nich, głosi, że co do zasady prawdopodobieństwo napotkania w korpusie kombinacji jakichś dwóch słów (np. "czerwona różna") jest produktem prawdopodobieństwa wystąpienia każdego z nich osobno ("czerwona" i "róża"). Jeżeli wystapień danej kombinacji jest więcej, niż wynikałoby to z tego założenia, oznacza to, że dana para słów jest dobrym kandydatem do kolokacji.

W tym notebooku zademonstruję jak obliczyć kilka najczęściej używanych miar kolokacji oraz wskażę na mocne i słabe strony każdej z nich.


## T-score

Pierwszą miarą, jakiej można użyć do oceny siły kolokacji jest $t$. W zasadzie obliczenie tej statystyki niczym nie różni się od obliczenia statystyki testowej w teście $t$ Studenta dla jednej próby. Załóżmy, że chcemy się dowiedzieć, czy zbitka "gorzka czekolady" jest dobrym kandydatem na kolokacje. Aby to ustalic potrzebujemy z korpusu wyekstraktować 4 informacje:
+ liczbę wszystkich wystąpień słowa "gorzka" w korpusie
+ liczbę wszystkich wystąpień słowa "czekolada" w korpusie
+ liczbę wystąpień zbitki "gorzka czekolada"
+ liczbę wszystkich słów zawartych w korpusie

Poniżej znajdują się dane zaczerpnięte z automatycznie wygenerowanego korpusu plTenTen dostępnego w SketchEngine (to nie jest do końca prawda, ale na użytek przykładu załóżmy, że są to prawdziwe liczby):

+ f(czekolada) = 135760
+ f(gorzka) = 8125
+ f(czekolada, gorzkiej) = 2824
+ wszystkie tokeny = 9387142186

Wzór na statystykę $t$ wygląda tak:

$$ t = \frac{\bar{x} - \mu}{\sqrt{\frac{s^2}{N}}} $$

Oznacza to, że musimy obliczyć róznicę między uzyskaną a spodziewaną wartością i podzielić ją przez błąd standardowy średniej. Skąd wziąć jednak wszystkie wartości, których wymaga ten wzór? Pierwsza wartośc jest dość prosta do uzyskania.

+ $\bar{x}$ - liczba wystąpień naszej zbitki podzielona na liczbę tokenów w korpusie ($w_1w_2 / N$), innymi słowy prawdopodobieństwo zbitki $w_1w_2$ oszacowane z próby

Jaka jest nasza wartość oczekiwana $\mu$? Jest to prawdopodobieństwo wystąpienia naszej zbitki ("gorzka czekolada") przy założeniu, że zdarzenia te (wystąpienie "gorzka" i "czekolada") są od siebie niezależne. Aby obliczyć to prawdopodobieństwo, musimy założyć, że wystąpienie zbitki w korpusie możemy modelować jako próbę Bernoulliego (np. tak jak rzut monetą). Zakładając, że występowanie "gorzka" i "czekolada" są od siebie niezależne, prawdopodobieństwo ich wystąpienia razem równe jest iloczynowi prawdopodobieńsw. Innymi słowy $P(gorzka\, czekolada) = P(gorzka)P(czekolada)$.

+ $\mu$ - prawdopodobieństwo wystapienia naszej zbitki w sytuacji, w ktorej wystapienie każdego słowa z osobna jest zdarzeniem niezależnym ($\frac{w_1}{N} \frac{w_2}{N}$)

We wzorze uwzględniamy również liczbe wszystkich słów w korpusie:

+ $N$ - liczba tokenów w korpusie

Ostatnią dana jest wariancja oznaczona symbolem $s^2$. Skoro przyjęliśmy, że wystąpienia słowa jest próba Bernouliego, to wariancja odpowiedniej zmiennej losowej (którą jest prawdopodobieństwo sukcesu) będzie równa $p(1-p)$.

Wykorzystując tę wiedzę spróbujmy obliczyć statystykę $t$ dla naszego przykładu.

In [48]:
f_czekolada = 135760 # liczba wystąpień "czekolada"
f_gorzka = 8125 # liczba wystąpień "gorzka"
f_czekolada_gorzka = 2824 # liczba wystąpień "gorzka czekolada"
n_tokens = 9387142186 # liczba wszystkich tokenów w korpusie

 # prawdopodobieństwo według hipotezy zerowej
p_czekolada_gorzka_h0 = (f_czekolada/n_tokens) * (f_gorzka/n_tokens)

# nasza estymacja prawdopodobieństwa z próby
p_czekolada_gorzka_ha = f_czekolada_gorzka/n_tokens 

print('Prawdopodobieństwo przy założeniu niezależności:')
print(p_czekolada_gorzka_h0)

print('\n')

print('Estymacja prawdopodobieństwa:')
print(p_czekolada_gorzka_ha)

Prawdopodobieństwo przy założeniu niezależności:
1.2517811479366614e-11


Estymacja prawdopodobieństwa:
3.0083703261805476e-07


Teraz, kiedy mamy już nasze $\bar{x}$ oraz $\mu$, możemy podstawić je do wzoru:

In [49]:
t_score = (p_czekolada_gorzka_ha - p_czekolada_gorzka_h0) /\
((p_czekolada_gorzka_ha * (1-p_czekolada_gorzka_ha))/n_tokens)**0.5
print('Uzyskany t-score:')
print(t_score)

Uzyskany t-score:
53.13911780816789


Oczywiście jeśli mamy statystykę $t$ to kuszącą propozycją może wydać się przetestowanie istotności naszego wyniku. Innymi słowy, chcemy zadac pytanie, czy mamy prawo odrzucić hipotezę zerową, głoszacą, że prawdziwe prawdopodobieństwo wystapienia tych dwóch słów obok siebie jest takie, jak wynikałoby to z hipotezy o niezalezności. Aby to zrobić musimy posłużyć się dystrybuantą rozkładu $t$ o $N-1$ stopniach swobody. Tak naprawdę nasze $N$ jest tak wysokie, że nie ma praktycznej różnicy między rozkładem $t$ a rozkladem normalnym, niemniej wykonamy test używając rozkładu $t$. W związku z faktem, że zwykle w korpusie mamy bardzo dużo danych, przyjmiemy zabójczy poziom istotności statystycznej $\alpha = 0.005$. Tak naprawdę, przy standardowym $\alpha = 0.05$ niemal zawsze w przypadku testów kolokacji odrzucimy hipotezę zerową, dlatego warto przyjać bardziej surowy próg. Łatwo zauważyć, że nasz t-score jest gigantyczny (region odrzucenia zaczyna się dla naszego progu istotności od 2.57, my otrzymaliśmy ponad 50!). 

Faktem, że zwykle odrzucimy hipotezę zerową nie powinniśmy się jednak przejmować. Znacznie cześciej niż do formalnych testów statystycznych miary omówione w tym rozdziale służą do oceny siły kolokacji (i tworzenia rankingu kolokatów).

In [13]:
from scipy.stats import t
print('Przy alpha = 0.005 odrzucamy H0 przy t wynoszacym:')
print(t(n_tokens-1).ppf(0.995))

print('\n')
print('P-value wynosi')

print(1 - t(n_tokens-1).cdf(t_score))

Przy alpha = 0.005 odrzucamy H0 przy t wynoszacym:
2.575829304072654


P-value wynosi
0.0


W praktyce statystyki $t$ nie stosuje się już zbyt często do pomiaru siły kolokacji. Zakłada ona rozkład normalny naszej zmiennej, co w przypadku danych korpusowych jest fałszem. Drugim kłopotem z $t$ jako miarą siły kolokacji jest to, że przecenia ona siłę kolokacji w sytuacji, w której mamy bardzo mało wystapień. Wciąż jednak w literaturze i oprogramowaniu można znaleźć tę statystykę. 

## $\chi^2$

Drugą często wykorzystywaną statystyką do pomiaru siły kolokacji jest klasycza statystyka $\chi^2$ i wykorzystujący ją test niezależności $\chi^2$. Wzór na tę statystyke wygląda tak:

$$\chi^2 = \sum_{i,j} \frac{(O_{ij} - E_{ij})^2}{E_{ij}}$$

+ O - zaobserwowana liczba wystąpień
+ E - oczekiwana liczba wystąpień

Test $\chi^2$ zazwyczaj przeprowadza się na tabelach kontyngencji, dlatego stwórzmy więc małą tabelkę, w której zestawimy nasze dane dotyczące frekwencji interesujących nas wyrażeń:

| X | $w_2 =$ czekolada | $w_2 \neq$ czekolada
|------|------| ---- |
|   $w_1 =$ gorzka  | 2824 | 8125 - 2824
|   $w_1 \neq$ gorzka  | 135760 - 2824| 9387142186 - 135760 - 8125


W zmiennej `observed` zapisaliśmy nasze 4 zaobserwowane wartości:

In [14]:
observed = [f_czekolada_gorzka, f_gorzka - f_czekolada_gorzka,
            f_czekolada - f_czekolada_gorzka, n_tokens - f_czekolada - f_gorzka]
observed

[2824, 5301, 132936, 9386998301]

Teraz musimy obliczyć wartości oczekiwane. Jak to zrobić? Najprostsza do zrozumienia metodą jest pomnożenie przez siebie sum brzegowych i podzielenie ich przez sumę zliczeń we wszystkich komórkach. Innymi słowy aby obliczyć wartość oczekiwaną dla zbitki "gorzka czekolada" musimy pomnożyć przez siebie (2824 + 5301) i (2824 + 132936), a uzyskany wynik podzielić przez 9386998301. Idea, leżąca u podstaw takiego postepowania, jest identyczna jak w poprzednim przykładzie - zakładamy, że nasze zmienne są niezależne i zgodnie z tym założeniem obliczamy odpowiednie wartości oczekiwane.

In [15]:
expected = [((observed[0] + observed[1]) * (observed[0] + observed[2])) / sum(observed),
            ((observed[0] + observed[1]) * (observed[1] + observed[3])) / sum(observed),
            ((observed[2] + observed[3]) * (observed[0] + observed[2])) / sum(observed),
            ((observed[2] + observed[3]) * (observed[1] + observed[3])) / sum(observed)
           ]

In [16]:
expected

[0.11750651156466765, 8124.882493488435, 135759.88249348843, 9386995477.117506]

Teraz pozostaje nam dla każdej komórki:
+ znaleźć różnice między wartością zaobserwowaną i oczekiwaną
+ podnieść tę różnice do kwadratu
+ podzielić ją przez wartość oczekiwaną
+ zsumować wszystkie wystandaryzowane reszty, które uzyskaliśmy

In [17]:
sum([((o - e) ** 2)/e for o,e in zip(observed, expected)])

67863767.39440794

Uzyskaliśmy gigantyczny wynik - nasza statystyka testowa wynosi ponad 67 milionów! Sprawdźmy, czy nie pomyliliśmy się w naszych obliczeniach importując odpowiedni moduł z pakietu NLTK (Natural Language Toolkit) i obliczmy tę samą statystykę: 

In [18]:
from nltk.collocations import BigramAssocMeasures
BigramAssocMeasures.chi_sq(observed[0], 
                           (observed[1] + observed[0], 
                            observed[2] + observed[0]), # tutaj podajemy sumy brzegowe
                           observed[3])

67862747.53303376

Okazuje się, że wszystkie nasze obliczenia były poprawne! Co za ulga. Oczywiście teraz, kiedy mamy obliczoną statystykę $\chi^2$ możemy wykonać test niezależności $\chi^2$ korzystając z faktu, że przy założeniu prawdziwości hipotezy zerowej statystyka ta ma rozkład $\chi^2$ o R-1 x C-1 stopniach swobody. My zdamy się na odpowiednią funkcję z modułu `scipy.stats`:

In [53]:
from scipy.stats import chi2_contingency
chisq, p, _, _ = chi2_contingency([[observed[0], observed[1]],
                                   [observed[2], observed[3]]])
print('Wartość chi-kwadrat:')
print(chisq)
print('\n')
print('Wartość p:')
print(p)

Wartość chi-kwadrat:
67839737.44290285


Wartość p:
0.0


$\chi^2$ jest miarą często używaną przez oprogramowanie do pracy z korpusami (np. wyszukiwarka Pelcra używa jej w swoim kolokatorze). Nie jest jednak pozbawiona wad. Podobnie jak $t$ kiepsko sprawdza się, gdy mamy mało zliczeń.

## Punktowa informacja wzajemna (*Pointwise Mutual Information*)

W przypadku tej miary ogólny koncept jest trochę inny niż poprzednio. Intuicyjnie rzecz ujmując PMI ma za zadanie powiedzieć nam, jak dużo dowiadujemy się o prawdopodobieństwie wystapienia jednego słowa ("gorzka") w sytuacji, gdy mamy napotykamy na drugie z nich ("czekolada"). Odpowiedni wzór jest dość prosty:

$$ I(x',y') = log_2 \frac{P(y'|x')}{P(y`)} $$

W naszym wypadku:

+ $P(y' | x')$ - prawdopodobieństwo wystapienia "gorzka czekolada" warunkowane na wystąpieniu "czekolada", w praktyce jest to po prostu liczba wystąpień tej kombinacji podzielona przez liczbę tokenów
+ $P(y')$ - prawdopodobieństwo wystąpienia "gorzka czekolada" przy założeniu, że wystapienie "gorzka" i "czekolada" sa od siebie niezależne

Oba prawdopodobieństwa liczyliśmy już przy omówieniu statystyki $t$. To co musimy zrobić, to wyciągnąć z ilorazu tych dwóch liczb logarytm przy podstawie 2.

In [20]:
f_czekolada = 135760
f_gorzka = 8125
f_gorzka_czekolada = 2824
n_tokens = 9387142186

from math import log
log(f_czekolada_gorzka/n_tokens / ((f_czekolada/n_tokens) * (f_gorzka/n_tokens)) , 2)

14.5527121969439

W literaturze często wskazuje się jednak, PMI nie jest optymalną miarą. Raczej nie zaleca się jej używania w celu "wyłapywania" potencjalnych kolokacji i oceny ich siły.

## Logarytmiczny wskaźnik wiarygodności (*log-likehood ratio*)

Ta miara jest nieco bardziej skomplikowana, w praktyce jednak jej obliczenie jest bardzo proste (bo redukuje się do tzw. testu G, o którym można przeczytać tu: https://en.wikipedia.org/wiki/G-test ). My jednak spróbujemy dość szczegółowo przejść przez wszystkie kroki obliczenia tej miary.

Sama miara została zaproponowana do oceny siły kolokacji przez Dunninga (1993). Przekonuje on, że ma ona kilka zalet w porównaniu z $t$ lub $\chi^2$:

1. Założeniem wielu testów jest to, że zmienne losowe mają rozkład w przybliżeniu normalny. Założenie to w przypadku danych korpusowych nie jest spełnione (szczególnie jeśli mamy małą liczbę wystąpień danego słowa).
2. W przypadku testu $\chi$ w rzeczywistości korzystamy z pewnego przybliżenia, które ma kiepskie własności w sytuacji, w której wariancja naszej zmiennej ($np(1-p)$ jeśli modelujemy liczbę wystąpień słowa w korpusie jako rozkład dwumianowy) jest niska (mniejsza niż 5).

Matematyczne szczegóły znajdują się (dla chętnych) w tekście Dunninga. 

Naszym zadaniem jest obliczenie współczynników wiarygodności (*likelihood*) dla dwóch hipotez a nastepnie obliczenie ich ilorazu (*likelihood ratio*). 

**Hipoteza 1** (niezależność). $P(w^2|w^1) = p = P(w^2|\neg w^1)$

**Hipoteza 2** (brak niezależności). $P(w^2|w^1) = p_1 \neq p_2 = P(w^2|\neg w^1)$

$p = \frac{c_2}{N}$

$p_1 = \frac{c_{12}}{N}$

$p_2 = \frac{c_2 - c_{12}}{N-c_1}$

gdzie:
+ $c_1$ - liczba wystapień pierwszego słowa
+ $c_2$ - liczba wystąpień drugiego słowa
+ $c_{12}$ - liczba wystapień zbitki
+ $N$ - liczba tokenów w korpusie 

Posiadając te informacje możemy obliczyć wiarygodność dla obu naszych hipotez (rozkład dwumianowy: $b(n,k,p)$ to prawdopodobienstwo $n$ sukcesów przy $k$ próbach o prawdopodobieństwie sukcesu $p$):

$L(H_1) = b(c_{12}; c_1, p) b(c_2 - c_{12}; N-c_1, p)$

$L(H_2) = b(c_{12}; c_1, p_1) b(c_2 - c_{12}; N-c_1, p_2) $ 

### Przykład 

Z powodu pewnej nieporęczności korzystania z powyższych wzorów przy dużych liczbach, jako kandydata na kolokacje weźmy wyrażenie "widzieć barwę" w skompilowanym przez nas korpusie percepcji. Odpowiednie dane to:

+ f(widzieć) = 1151
+ f(barwę) = 56
+ f(widzieć barwę) = 12
+ liczba tokenów = 427,659

Przy obliczaniu skorzystamy z funkcji pozwalającej pracować z rozkładem dwumianowym `scipy.stats.binom.pmf`. 

In [54]:
from scipy.stats import binom
from math import log

# Dane z przykładu
w1 = 1151
w2 = 56
w1w2 = 12
tokens = 427659

# Obliczamy niezbędne prawdopodobieństwa
p = w2 / tokens
p1 = w1w2 / w1
p2 = (w2 - w1w2) / (tokens - w1)

# wiarygodność dla h1 i h2
lh1 = binom.pmf(w1w2, w1, p) * binom.pmf(w2 - w1w2, tokens - w1, p)
lh2 = binom.pmf(w1w2, w1, p1) * binom.pmf(w2 - w1w2, tokens - w1, p2)

# Obliczamy logarytm z ilorazu i mnożymy przez -2
loglik = -2 * log(lh1/lh2)
loglik

84.19165242130423

Wiemy, że tak obliczona statystyka ($-2log\frac{L(H_1)}{L(H_2)}$) ma rozkład $\chi^2$ o jednym stopniu swobody, możemy więc wykonać test statystyczny:

In [55]:
from scipy.stats import chi2

pval = 1 - chi2.cdf(loglik, 1)

print('Wartość p wynosi:')
print(pval) 

Wartość p wynosi:
0.0


Łatwo zauważyć, że wartość $p$ jest tak niska, że Python zaokrągla ją do 0. W przypadku wiekszych korpusów trudno obliczyć tę miarę w podany wyżej sposób, niemniej jest to wciąż możliwe. Za pomocą funkcji z pakietu NLTK obliczmy więc tę miarę dla naszego poprzedniego przykładu z czekoladą:

In [23]:
BigramAssocMeasures.likelihood_ratio(observed[0], 
                                     (observed[1] + observed[0],
                                      observed[2] + observed[0]),
                                     observed[3])

52503.975041350015

## logDice

logDice jest miarą siły kolokacji, która swoje zastosowanie ma znaleźć szczególnie w pracy leksykografa. Rychly (2008) wskazuje na kilka problemów związanych ze standardowymi miarami siły kolokacji:

1. Miary te są mocno dostosowane do konkretnej wielkości korpusu - jeśli użyjemymy ich w innym korpusie z inną liczbą tokenów, to lista kolokatów moze nie być satysfakcjonująca.
2. Miary te są zazwyczaj pozbawione intuicyjnego sensu i nie można ich użyć np. do porównywania wyników z dwóch korpusów lub korpusu o podkorpusów, a zwykle to jest celem użytkownikóœ korpusów zainteresowanych miarami siły kolokacji.

Opracowana przez niego miara *logDice* ma kilka bardzo przydatnych własności:

1. Teoretyczne maksimum miary *logDice* wynosi 14 i taką wartość przyjmuje w sytuacji, w której dane słowo X zawsze występuje ze słowem Y i odwrotnie. Zwykle *logDice* wynosi mniej niż 10.
2. Wartość 0 oznacza, że jest mniej niż 1 współwystąpienie zbitki XY na 16 000 wystąpień X lub 16 000 wystąpień Y. Ujemne wartości oznaczają, że nie występuje statystyczna istotność kolokacji XY (ale nie uzasadnia tego w artykule).
3. Jeśli porównamy dwa wyniki *logDice* dwóch kolokacji to różnica jednego punktu oznacza dwukrotnie częstsze występowanie danej kolokacji a różnica 7 punktów około 100 krotną różnicę.
4. Wynik nie jest zależny od wielkości korpusu, uwzględnia tylko częstość XY relatywną do częstości X i Y.


Wzór jest wyjątkowo prosty, a sama miara w praktyce bardzo dobrze sprawdza się w wyłapywaniu silnych zwiazków międzywyrazowych zarówno w dużych jak i mniejszych korpusach. Jest to standardowa miara używana w funkcji WordSketch w SketchEngine.

$$ Dice = \frac{f_(AB)}{f_A + f_B}$$

$$ logDice = 14 + log_2(Dice) $$

Obliczmy logDice dla naszego przykładu "widzieć barwę" z korpusu percepcji:

In [24]:
dice = (2 * w1w2)/(w1 + w2)
logdice = 14 + log(dice, 2)
logdice

8.347752539966134

# Bibliografia

Manning, C. D., Manning, C. D., & Schütze, H. (1999). *Foundations of statistical natural language processing*. MIT press.

Dunning, T. (1993). Accurate methods for the statistics of surprise and coincidence. *Computational linguistics*, 19(1), 61-74.

Rychlý, P. (2008). A lexicographer-friendly association score. *Proceedings of Recent Advances in Slavonic Natural Language Processing*, RASLAN, 2008, 6.

Implementacja wszystkich tu omówionych (i kilku innych) miar siły kolokacji znajduje się w Pythonowskim pakiecie NLTK: http://www.nltk.org/api/nltk.metrics.html#nltk.metrics.association.NgramAssocMeasures